# Feature extraction

Load the data from the HDF5 files (it weights much less than the initial files).

In [1]:
import pandas as pd

df = pd.concat(
    (
        pd.read_csv('data/training_set_metadata.csv'),
        pd.read_csv('data/test_set_metadata.csv')
    ),
    ignore_index=True,
    sort=False
)
df['is_train'] = df['target'].notnull()

Polar coordinates.

In [2]:
df['rho'] = df.eval('sqrt(gal_l ** 2 + gal_b ** 2)')
df['phi'] = df.eval('arctan2(gal_b, gal_l)')

Photometric redshift.

In [3]:
df['hostgal_photoz_min'] = df.eval('hostgal_photoz - hostgal_photoz_err')
df['hostgal_photoz_max'] = df.eval('hostgal_photoz + hostgal_photoz_err')
df['hostgal_photoz_over_err'] = df.eval('hostgal_photoz / (hostgal_photoz_err + 1)')

Load light curve features.

In [5]:
import numpy as np

with pd.HDFStore('data/features.h5') as store:
    for key in store:
        df = df.join(store.get(key).astype(np.float32), on='object_id')

Build computed features.

In [6]:
#('u', 'g', 'r', 'i', 'z', 'y')

for i in range(6):
    
    for p1, p2 in [(95, 5), (90, 10), (75, 25), (50, 10), (90, 50)]:
        df[f'flux_p{p1}_minus_p{p2}_{i}'] = df.eval(f'flux_p{p1}_{i} - flux_p{p2}_{i}')
        
    df[f'flux_max_minus_p95_{i}'] = df.eval(f'flux_max_{i} - flux_p95_{i}')
    df[f'flux_p95_minus_min_{i}'] = df.eval(f'flux_p95_{i} - flux_min_{i}')
    
    df[f'flux_count_ratio_{i}'] = df.eval(f'flux_count_above_mean_{i} / flux_count_below_mean_{i}') 
    
    df[f'flux_mean_over_flux_err_mean_{i}'] = df.eval(f'flux_mean_{i} / flux_err_mean_{i}') 
    df[f'flux_mean_over_flux_std_{i}'] = df.eval(f'flux_mean_{i} / flux_std_{i}') 
    
    df[f'flux_fft_amp_0_over_fft_amp_1_{i}'] = df.eval(f'flux_fft_amp_0_{i} / flux_fft_amp_1_{i}') 
    df[f'flux_fft_amp_0_over_fft_amp_2_{i}'] = df.eval(f'flux_fft_amp_0_{i} / flux_fft_amp_2_{i}')

Interactions between passbands.

In [7]:
import itertools

for (i, j) in itertools.combinations(range(6), 2):
    for stat in ('mean', 'min', 'max', 'std', 'skew', 'kurtosis'):
        df[f'flux_{stat}_{i}_over_{j}'] = df.eval(f'flux_{stat}_{i} / (flux_{stat}_{j} + 1)')

See what we got.

In [8]:
df.head()

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,...,flux_max_3_over_5,flux_std_3_over_5,flux_skew_3_over_5,flux_kurtosis_3_over_5,flux_mean_4_over_5,flux_min_4_over_5,flux_max_4_over_5,flux_std_4_over_5,flux_skew_4_over_5,flux_kurtosis_4_over_5
0,615,349.046051,-61.943836,320.796530,-51.753706,1,0.0000,0.0000,NaN,0.017,...,1.175504,1.134038,0.260496,2.445696,1.204623,1.000876,1.007293,0.986557,0.177820,2.611787
1,713,53.085938,-27.784405,223.525509,-54.460748,1,1.6267,0.2552,45.4063,0.007,...,0.718432,0.796930,-0.137234,-5.631334,1.133582,0.930032,0.623169,0.791566,-0.052647,-5.864635
2,730,33.574219,-6.579593,170.455585,-61.548219,1,0.2262,0.0157,40.2561,0.021,...,0.694930,0.571557,0.993181,1.554760,0.777834,0.321386,0.851996,0.747263,0.946817,1.326837
3,745,0.189873,-45.586655,328.254458,-68.969298,1,0.2813,1.1523,40.7951,0.007,...,1.426188,1.292222,0.899480,1.248039,1.118192,1.675188,1.288533,1.222058,0.833502,1.046656
4,1124,352.711273,-63.823658,316.922299,-51.059403,1,0.2415,0.0176,40.4166,0.024,...,1.269258,1.170987,0.773668,0.913272,1.254454,1.677773,1.303589,1.197520,0.838255,1.061070


# Learning

In [10]:
to_drop = ['is_train', 'target', 'hostgal_specz']

train = df[df['is_train']].set_index('object_id')
test = df[~df['is_train']].set_index('object_id')

X_train = train.drop(columns=to_drop)
y_train = train['target'].apply(lambda x: f'class_{int(x)}').astype('category')
X_test = test.drop(columns=to_drop)

submission = pd.DataFrame(0.0, index=test.index, columns=y_train.cat.categories)
submission['class_99'] = 0.0

class_weights = {c: 1 for c in y_train.cat.categories}
class_weights['class_64'] = 2
class_weights['class_15'] = 2

In [11]:
assert len(X_train.columns) == len(X_test.columns)
assert len(X_train) == len(y_train)
assert len(X_test) == 3492890
assert len(submission) == 3492890

In [12]:
def make_metric(class_to_int):

    def metric(y_pred, y_true):
        """
        @author olivier https://www.kaggle.com/ogrellier
        multi logloss for PLAsTiCC challenge
        """
        class_weight = {
            class_to_int[k]: v
            for k, v in class_weights.items()
            if k in class_to_int
        }
        
        y_true = y_true.get_label()    
        y_p = y_pred.reshape(y_true.shape[0], len(class_weight), order='F')
        
        # Trasform y_true in dummies
        y_ohe = pd.get_dummies(y_true)
        # Normalize rows and limit y_preds to 1e-15, 1-1e-15
        y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
        # Transform to log
        y_p_log = np.log(y_p)
        # Get the log for ones, .values is used to drop the index of DataFrames
        # Exclude class 99 for now, since there is no class99 in the training set
        # we gave a special process for that class
        y_log_ones = np.sum(y_ohe.values * y_p_log, axis=0)
        # Get the number of positives for each class
        nb_pos = y_ohe.sum(axis=0).values.astype(float)
        # Weight average and divide by the number of positives
        class_arr = np.array([class_weight[k] for k in sorted(class_weight.keys())])
        y_w = y_log_ones * class_arr / nb_pos

        loss = - np.sum(y_w) / np.sum(class_arr)
        return 'wloss', loss, False
    
    return metric

## Galactic objects

Select the galactic objects.

In [13]:
X_train_gal = X_train[X_train['hostgal_photoz'] == 0]
y_train_gal = y_train[X_train['hostgal_photoz'] == 0]
X_test_gal = X_test[X_test['hostgal_photoz'] == 0]

class_to_int = {c: i for i, c in enumerate(y_train_gal.unique())}
int_to_class = {i: c for c, i in class_to_int.items()}

Train the model.

In [14]:
import lightgbm as lgbm
import numpy as np
from sklearn import model_selection


params = {
    'application': 'multiclass',
    'boosting_type': 'gbdt',
    'num_classes': y_train_gal.nunique(),
    'metric': 'None',
    'num_threads': 8,
    'num_leaves': 2 ** 3,
    'max_bin': 255,
    'min_data_in_bin': 40,
    'min_data_in_leaf': 300,
    'min_sum_hessian_in_leaf': 5e-2,
    'learning_rate': 0.07,
    'feature_fraction': 0.7,
    'feature_fraction_seed': 42,
    'bagging_fraction': 1,
    'bagging_freq': 0,
    'bagging_seed': 42,
    'lambda_l1': 0.5,
    'lambda_l2': 0.1,
    'verbosity': 2,
}
        

cv = model_selection.KFold(n_splits=5, shuffle=True, random_state=42)
feature_importances = pd.DataFrame(index=X_train_gal.columns)
gal_fit_scores = np.zeros(cv.n_splits)
gal_val_scores = np.zeros(cv.n_splits)
submission.loc[X_test_gal.index, y_train_gal.unique()] = 0.0

for i, (fit_idx, val_idx) in enumerate(cv.split(X_train_gal, y_train_gal)):
    
    X_fit = X_train_gal.iloc[fit_idx]
    y_fit = y_train_gal.iloc[fit_idx].map(class_to_int)
    w_fit = y_train_gal.iloc[fit_idx].map(class_weights)
    X_val = X_train_gal.iloc[val_idx]
    y_val = y_train_gal.iloc[val_idx].map(class_to_int)
    w_val = y_train_gal.iloc[val_idx].map(class_weights)
    
    # Train the model
    fit_set = lgbm.Dataset(X_fit, y_fit, weight=w_fit)
    val_set = lgbm.Dataset(X_val, y_val, reference=fit_set, weight=w_val)

    evals_result = {}
    model = lgbm.train(
        params=params,
        train_set=fit_set,
        feval=make_metric(class_to_int),
        num_boost_round=10000,
        valid_sets=(fit_set, val_set),
        valid_names=('fit', 'val'),
        verbose_eval=50,
        early_stopping_rounds=80,
        evals_result=evals_result
    )
    
    # Store the feature importances
    feature_importances[f'gain_{i}'] = model.feature_importance('gain')
    feature_importances[f'split_{i}'] = model.feature_importance('split')
    
    # Store the predictions
    y_pred = pd.DataFrame(model.predict(X_test_gal), index=X_test_gal.index)
    y_pred.columns = y_pred.columns.map(int_to_class)
    submission.loc[y_pred.index, y_pred.columns] += y_pred / cv.n_splits
    
    # Store the scores
    gal_fit_scores[i] = evals_result['fit']['wloss'][-1]
    gal_val_scores[i] = evals_result['val']['wloss'][-1]

print(f'- Train loss: {gal_fit_scores.mean():.3f} (±{gal_fit_scores.std():.3f})')
print(f'- Valid loss: {gal_val_scores.mean():.3f} (±{gal_val_scores.std():.3f})')

Training until validation scores don't improve for 80 rounds.
[50]	fit's wloss: 0.601887	val's wloss: 0.683714
[100]	fit's wloss: 0.208818	val's wloss: 0.344999
[150]	fit's wloss: 0.0812844	val's wloss: 0.252062
[200]	fit's wloss: 0.0415634	val's wloss: 0.228031
[250]	fit's wloss: 0.0262068	val's wloss: 0.216909
[300]	fit's wloss: 0.0196423	val's wloss: 0.214341
[350]	fit's wloss: 0.0161115	val's wloss: 0.213019
[400]	fit's wloss: 0.0140102	val's wloss: 0.212127
[450]	fit's wloss: 0.0128415	val's wloss: 0.212569
Early stopping, best iteration is:
[415]	fit's wloss: 0.0135037	val's wloss: 0.211173
Training until validation scores don't improve for 80 rounds.
[50]	fit's wloss: 0.590446	val's wloss: 0.595809
[100]	fit's wloss: 0.202028	val's wloss: 0.245451
[150]	fit's wloss: 0.0830039	val's wloss: 0.148036
[200]	fit's wloss: 0.0429107	val's wloss: 0.123731
[250]	fit's wloss: 0.0269552	val's wloss: 0.119428
[300]	fit's wloss: 0.0197163	val's wloss: 0.120393
[350]	fit's wloss: 0.0154324	va

- Train loss: 0.011 (±0.005)
- Valid loss: 0.170 (±0.042)

In [15]:
feature_importances.sort_values('gain_0', ascending=False)

,gain_0,split_0,gain_1,split_1,gain_2,split_2,gain_3,split_3,gain_4,split_4
flux_skew_2,5317.219884,106,5279.706899,115,5511.060142,138,5535.871414,101,5314.917607,109
flux_p50_minus_p10_2,3112.163856,70,3017.696433,92,2375.109735,8,1853.948162,11,2749.767235,38
flux_p50_minus_p10_1,3040.159070,34,3078.306712,45,3269.022793,35,3940.889268,31,3524.050522,40
flux_p75_minus_p25_2,1472.726760,99,1151.402920,45,2102.379730,88,1629.613339,28,1422.707921,67
flux_skew_3,1298.911371,93,1606.658832,109,1376.874025,88,1605.119122,88,1627.254659,124
flux_skew_1,1095.559804,121,789.540452,99,898.096942,60,1099.295070,42,909.655234,102
flux_p75_minus_p25_1,881.825727,12,1336.990432,28,991.489664,12,1398.494204,31,1606.656728,35
flux_min_0_over_2,827.786226,37,899.320145,54,1548.201163,55,949.256187,30,1186.074511,96
detected_mean_5,715.062673,61,648.540729,80,1327.385989,72,661.754796,40,1264.230087,91
flux_p90_minus_p10_2,694.340251,14,897.718768,20,317.856261,35,419.717535,18,390.534402,9


## Extragalactic objects

Select the extragalactic objects.

In [16]:
X_train_ex = X_train[X_train['hostgal_photoz'] > 0]
y_train_ex = y_train[X_train['hostgal_photoz'] > 0]
X_test_ex = X_test[X_test['hostgal_photoz'] > 0]

class_to_int = {c: i for i, c in enumerate(y_train_ex.unique())}
int_to_class = {i: c for c, i in class_to_int.items()}

In [17]:
import lightgbm as lgbm
import numpy as np
from sklearn import model_selection


params = {
    'application': 'multiclass',
    'boosting_type': 'gbdt',
    'num_classes': y_train_ex.nunique(),
    'metric': 'None',
    'num_threads': 8,
    'num_leaves': 2 ** 3,
    'max_bin': 255,
    'min_data_in_bin': 40,
    'min_data_in_leaf': 420,
    'min_sum_hessian_in_leaf': 2e-4,
    'learning_rate': 0.07,
    'feature_fraction': 0.75,
    'feature_fraction_seed': 42,
    'bagging_fraction': 1,
    'bagging_freq': 0,
    'bagging_seed': 42,
    'lambda_l1': 0.8,
    'lambda_l2': 0.2,
    'verbosity': 2,
}
        

cv = model_selection.KFold(n_splits=5, shuffle=True, random_state=42)
feature_importances = pd.DataFrame(index=X_train_ex.columns)
ex_fit_scores = np.zeros(cv.n_splits)
ex_val_scores = np.zeros(cv.n_splits)
submission.loc[X_test_ex.index, y_train_ex.unique()] = 0.0

for i, (fit_idx, val_idx) in enumerate(cv.split(X_train_ex, y_train_ex)):
    
    X_fit = X_train_ex.iloc[fit_idx]
    y_fit = y_train_ex.iloc[fit_idx].map(class_to_int)
    w_fit = y_train_ex.iloc[fit_idx].map(class_weights)
    X_val = X_train_ex.iloc[val_idx]
    y_val = y_train_ex.iloc[val_idx].map(class_to_int)
    w_val = y_train_ex.iloc[val_idx].map(class_weights)
    
    # Train the model
    fit_set = lgbm.Dataset(X_fit.values.astype(np.float32), y_fit, weight=w_fit)
    val_set = lgbm.Dataset(X_val.values.astype(np.float32), y_val, reference=fit_set, weight=w_val)
    
    evals_result = {}
    model = lgbm.train(
        params=params,
        train_set=fit_set,
        feval=make_metric(class_to_int),
        num_boost_round=10000,
        valid_sets=(fit_set, val_set),
        valid_names=('fit', 'val'),
        verbose_eval=50,
        early_stopping_rounds=80,
        evals_result=evals_result
    )
    
    # Store the feature importances
    feature_importances[f'gain_{i}'] = model.feature_importance('gain')
    feature_importances[f'split_{i}'] = model.feature_importance('split')
    
    # Store the predictions
    y_pred = pd.DataFrame(model.predict(X_test_ex.values.astype(np.float32)), index=X_test_ex.index)
    y_pred.columns = y_pred.columns.map(int_to_class)
    submission.loc[y_pred.index, y_pred.columns] += y_pred / cv.n_splits
    
    # Store the scores
    ex_fit_scores[i] = evals_result['fit']['wloss'][-1]
    ex_val_scores[i] = evals_result['val']['wloss'][-1]

print(f'- Train loss: {ex_fit_scores.mean():.3f} (±{ex_fit_scores.std():.3f})')
print(f'- Valid loss: {ex_val_scores.mean():.3f} (±{ex_val_scores.std():.3f})')

Training until validation scores don't improve for 80 rounds.
[50]	fit's wloss: 1.28889	val's wloss: 1.47817
[100]	fit's wloss: 0.87873	val's wloss: 1.2269
[150]	fit's wloss: 0.644339	val's wloss: 1.13621
[200]	fit's wloss: 0.490329	val's wloss: 1.10574
[250]	fit's wloss: 0.38385	val's wloss: 1.10629
[300]	fit's wloss: 0.306504	val's wloss: 1.12371
Early stopping, best iteration is:
[228]	fit's wloss: 0.426306	val's wloss: 1.10206
Training until validation scores don't improve for 80 rounds.
[50]	fit's wloss: 1.28165	val's wloss: 1.43122
[100]	fit's wloss: 0.875724	val's wloss: 1.1944
[150]	fit's wloss: 0.644793	val's wloss: 1.11079
[200]	fit's wloss: 0.49315	val's wloss: 1.07729
[250]	fit's wloss: 0.388416	val's wloss: 1.06542
[300]	fit's wloss: 0.312604	val's wloss: 1.06506
Early stopping, best iteration is:
[260]	fit's wloss: 0.371106	val's wloss: 1.06199
Training until validation scores don't improve for 80 rounds.
[50]	fit's wloss: 1.27411	val's wloss: 1.46502
[100]	fit's wloss: 0

- Train loss: 0.380 (±0.028)
- Valid loss: 1.139 (±0.023)

In [23]:
feature_importances.sort_values('gain_0', ascending=False)

,gain_0,split_0,gain_1,split_1,gain_2,split_2,gain_3,split_3,gain_4,split_4
distmod,4499.036833,230,3792.133100,204,4105.395287,175,4552.744083,204,3596.797351,213
flux_std_0_over_4,3450.781603,34,1710.364334,50,3797.270123,11,2350.579467,32,2695.968090,41
hostgal_photoz,3000.366893,195,3515.950076,204,2951.162492,134,2668.127031,156,3984.972353,186
detected_mean_1,2863.543513,69,2979.774085,63,2609.628314,46,2629.016639,68,2598.381418,60
flux_p50_minus_p10_1,2648.340814,26,2683.767757,27,2355.288087,20,3220.233419,26,2950.077004,14
flux_p50_minus_p10_2,1926.065930,31,1658.411686,34,1075.671698,12,1094.068193,9,1529.565455,22
flux_mean_0_over_3,1911.197456,68,1869.272470,66,1575.537990,40,1295.134050,41,1527.701534,37
flux_max_2_over_5,1701.154187,88,1522.614518,116,1750.845599,75,1939.225837,128,1812.854352,111
hostgal_photoz_min,1686.397954,205,2474.668019,253,1811.107849,170,2087.061920,172,2279.539387,166
flux_mean_0_over_4,1673.947224,20,1750.615208,37,1141.683636,16,1304.763378,14,1111.087471,28


## Class 99

In [19]:
submission['class_99'] = (1 - submission[submission.columns.drop('class_99')].max(axis='columns')) / 2.3

## Saving the submission

In [20]:
submission.head()

,class_15,class_16,class_42,class_52,class_53,class_6,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
object_id,,,,,,,,,,,,,,,
13,0.001847,0.0,0.455465,0.126309,0.0,0.0,0.024366,0.000409,0.0,0.007082,0.000522,0.380723,0.0,0.003278,0.236754
14,0.007257,0.0,0.181782,0.014037,0.0,0.0,0.019633,0.003569,0.0,0.003568,0.005350,0.756907,0.0,0.007897,0.105693
17,0.002663,0.0,0.088851,0.010028,0.0,0.0,0.022678,0.002425,0.0,0.009998,0.003336,0.821194,0.0,0.038827,0.077742
23,0.001067,0.0,0.072688,0.009849,0.0,0.0,0.031704,0.001074,0.0,0.057629,0.011708,0.807748,0.0,0.006534,0.083588
34,0.000606,0.0,0.043093,0.047223,0.0,0.0,0.010707,0.000241,0.0,0.004203,0.000210,0.893409,0.0,0.000309,0.046344


Sanity checks.

In [21]:
assert submission[X_test['hostgal_photoz'] == 0][y_train_ex.unique().categories].sum().sum() == 0
assert submission[X_test['hostgal_photoz'] > 0][y_train_gal.unique().categories].sum().sum() == 0

Save the submission. We align with the sample submission just to make sure.

In [22]:
name = f'{gal_val_scores.mean():.3f}_{gal_val_scores.std():.3f}_{ex_val_scores.mean():.3f}_{ex_val_scores.std():.3f}'

sample_sub = pd.read_csv('data/sample_submission.csv').set_index('object_id')

submission.loc[sample_sub.index, sample_sub.columns].to_csv(f'submissions/{name}.csv.gz', compression='gzip')